# Tutorial: CoLa and CoSo

CoLa and CoSo provide a framework for solving combinatorial counting problems. CoLa is a language to declare simple combinatorics problems, CoSo is a solver for CoLa and provides an explanation of the solution. 
The content of this tutorial is based on the publication:
> *Pietro Totis, Jesse Davis, Luc De Raedt, Angelika Kimmig: <br>
> Lifted Reasoning for Combinatorial Counting. J. Artif. Intell. Res. 76: 1-58 (2023)*

## CoLa


In this tutorial we consider the following setup:
> A kit of toy shapes contains five triangles and two squares. One triangle and one square are red. Another triangle and the other square are blue, and the remaining triangles are green. 

And solve the following problems:

> **P1**: In how many different rows of four objects can the shapes be arranged if the two squares are included and the second object is green?

> **P2**: In how many ways can the objects be divided into three(non-empty) groups such that the green objects all belong to the same group?

Three components define a CoLa problem:
1. A (multi)set of objects, e.g. the toy shapes
2. A configuration, e.g. the row
3. Constraints, e.g. two squares are included
    

### 1. Objects

In CoLa the (multi)set of objects is called *universe* and *properties* are subsets of the universe denoting a group of objects that have something in common.
Objects are declared in two ways: explicitly or implicitly.

#### Explicit
The explicit declaration of universe and properties corresponds to the enumeration of the objects. Repeated/identical objects (if a multiset) are expressed by repeating the label as many times as the number of identical copies of the object.

    property shapes = {square_red, square_blue, 
                       triangle_red, triangle_blue, 
                       triangle_green, triangle_green, 
                       triangle_green
                       };
    property red = {triangle_red, square_red};
    property blue = {triangle_blue, square_blue};
    property green = {triangle_green};
    property triangle = {triangle_red, triangle_blue, triangle_green};
    property square = {square_blue, square_red};

#### Implicit
The implicit declaration corresponds to the definition of the size of each property (cf. constraints) and their intersections:

    property red;
    #red=2;
    property blue;  
    #blue=2;
    property green;
    #green=3;

    property triangle; 
    #triangle=7;
    property square;
    #square=2;

    #square&red=1; 
    #square&blue=1; 
    #triangle&red=1;
    #triangle&blue=1;  
    #triangle&green=3;

Base properties can be composed with set operations: and (`&`), or (`+`), complement (`¬`). The complement is computed with respect to the universe.

### 2. Configuration
Configurations define how objects should be arranged. CoLa offers six types of configurations: [sequences](https://en.wikipedia.org/wiki/Sequence), [permutations](https://en.wikipedia.org/wiki/Permutation), [multisubsets](https://en.wikipedia.org/wiki/Multiset#Counting_multisets), [subset](https://en.wikipedia.org/wiki/Combination), [partition](https://en.wikipedia.org/wiki/Partition_(number_theory)) and [composition](https://en.wikipedia.org/wiki/Composition_(combinatorics)).

A set of possible configurations is specified by the (multi)set of objects, e.g. *shapes*, and its type. The type is expressed as follows:
1. sequences: `[repeated shapes]`
2. permutations: `[shapes]`
3. multisubsets: `{repeated shapes}`
4. subsets: `{shapes}`
5. partitions: `{{shapes}}`
6. compositions: `[{shapes}]`

In CoLa a label associated to the configuration denotes any of the possible configurations with the keyword `in`:

    row in [shapes];
    groups in {{shapes}};

### 3. Constraints
Constraints in CoLa are of three types:
1. Size constraints
2. Counting constraints
3. Positional constraints

#### 3.1 Size constraints
Size constraints define the number of objects in a group or in a configuration.
Implicit declarations of the objects are an example of size constraints where the comparison operator is an equality:

    #red=2;  #blue=2;  #green=3;
    
and the size of the configuration can be defined in the same way:

    row in [shapes];
    #row = 4;

    groups in {{shapes}};
    #groups = 3;
but all the other typical comparison operators can be used to specify the size of the configurations, for instance:
    #row < 5;
    #groups != 4;
#### 3.2 Counting constraints
Counting constraints count the number of objects with some property in a configuration or a level 2 group.
All comparison operators can also be used to describe the desired number.

    #(row & square) = 2;
    #(row & square) < 4;
    #(row & square) != 1;
    
With level 2 configurations the constraints can be nested, either counting constraints or size constraints, the keyword *part* can be used to refer to a generic group of the configuration. For example:

    #( #part & green = 3 ) = 1;
    
#### 3.3 Positional constraints
Positional constraints apply only to ordered configurations and are used to denote a specific position in the configuration. The position can be assigned a specific property depending on the type of the position, that is, an object in level 1 configurations or a group in level 2 configurations.
In level 1 configurations a (level 1) property can be assigned to a specific position with the = operator:

    row[2] = green;
In compositions (ordered level 2 configurations) both size and counting constraints can be used to express a property of a group in a given position, for example:

    pos_groups in [{shapes}];

    #pos_groups[1] >= 3;
    #( pos_groups[2]&triangles ) <= 2;

## CoSo

CoSo can be installed via pip as a Python package with:

    pip install coso
    
The source code is available on [Github](https://github.com/PietroTotis/CoSo).
The solver can be then called with the function `coso`, either with the parameter `file=*path_to_file*` to run CoSo on a file containing a CoLa problem, or with the parameter `cola=*string*` to give the solver a python string with the problem encoding, as in the foolowing example:

In [3]:
from coso.launcher import run

problem_1 = """
    property shapes = {square_red, square_blue, 
                       triangle_red, triangle_blue, 
                       triangle_green, triangle_green, 
                       triangle_green
                       };
    property red = {triangle_red, square_red};
    property blue = {triangle_blue, square_blue};
    property green = {triangle_green};
    property triangle = {triangle_red, triangle_blue, triangle_green};
    property square = {square_blue, square_red};

    
    row in [shapes];
    #row = 4;
    
    row[2] = green;
    
    #(row & square) = 2;
"""
solution = run(cola=problem_1)
print(solution)

18 (8 subproblems)


CoSo can also generate visual descriptions of the solving procedures with the function `viscoso` which takes the same arguments as `coso` with an additional parameter `visual=*path_to_file*` which specifies an html file where the visual representation should be written. Then the file can be opened in a web browser.
In a jupyter notebook the visual interface is provided with a widget:

In [6]:
from coso.VisCoSo_widget import viscoso

viscoso()

HTML(value='\n            <script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js…

Button(description='Solve', icon='check', style=ButtonStyle(), tooltip='Solve')

FileNotFoundError: [Errno 2] No such file or directory: '/media/windows/Users/pietr/Desktop/PhD/CoSo/pyenv/lib/python3.8/site-packages/src/VisCoSo/css/viscoso.css'

```{raw} html
:file: src/VisCoSo/prova.html
```